In [1]:
import tensorflow.keras.backend as K
import tensorflow as tf
from tensorflow.keras import backend as K
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from patchify import patchify, unpatchify
from tensorflow.keras import backend as K
from skimage.measure import label

def f1(y_true, y_pred, threshold=0.3):
    y_pred = tf.cast(y_pred > threshold, tf.float32)
    TP = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
    Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
    Pred_Positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
    precision = TP / (Pred_Positives + tf.keras.backend.epsilon())
    recall = TP / (Positives + tf.keras.backend.epsilon())
    return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
    
@tf.keras.utils.register_keras_serializable()
def f1_metric(y_true, y_pred):
    return f1(y_true, y_pred, threshold=0.3)


def weighted_binary_crossentropy(y_true, y_pred):
    """
    Weighted binary cross-entropy to address class imbalance.
    """
    # Define weights for foreground (root, shoot, seed) and background
    weight_foreground = 10.0
    weight_background = 1.0

    # Compute weighted binary cross-entropy
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    weights = tf.where(y_true == 1, weight_foreground, weight_background)
    loss = K.binary_crossentropy(y_true, y_pred)
    weighted_loss = loss * weights
    return K.mean(weighted_loss)
def dice_loss(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + 1e-7) / (union + 1e-7)
    return 1 - dice

@tf.keras.utils.register_keras_serializable()
def combined_loss(y_true, y_pred):
    return 0.5 * dice_loss(y_true, y_pred) + 0.5 * weighted_binary_crossentropy(y_true, y_pred)





2025-01-09 08:34:47.725911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-09 08:34:47.725966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-09 08:34:47.727515: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 08:34:47.736661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import logging

# Logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("pipeline_log.txt"), logging.StreamHandler()],
)

def preprocess_image(image_path):
    """Load and preprocess the input image."""
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Failed to load image: {image_path}")
    return image

def extract_petri_dish(image):
    """Extract the largest contour assumed to be the Petri dish."""
    _, thresholded = cv2.threshold(image, 57, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        logging.warning("No contours detected for Petri dish.")
        return image, np.ones_like(image)

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

def predict_root_mask(image, model, patch_size=128, stride=64):
    """Predict root mask using a patch-based model."""
    h, w = image.shape
    patches = []
    positions = []
    for y in range(0, h - patch_size + 1, stride):
        for x in range(0, w - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size]
            patch_rgb = np.stack([patch]*3, axis=-1)
            patches.append(patch_rgb)
            positions.append((y, x))

    patches = np.array(patches) / 255.0
    predictions = model.predict(patches, verbose=0)

    reconstructed = np.zeros((h, w), dtype=np.float32)
    counts = np.zeros((h, w), dtype=np.float32)
    for pred, (y, x) in zip(predictions, positions):
        pred = pred[..., 0]
        reconstructed[y:y+patch_size, x:x+patch_size] += pred
        counts[y:y+patch_size, x:x+patch_size] += 1
    return (reconstructed / np.maximum(counts, 1) > 0.3).astype(np.uint8)

def connect_roots(mask):
    """Dilate the mask to connect fragmented roots."""
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    return cv2.dilate(mask, kernel, iterations=27)

def filter_and_select_largest_objects(mask, min_area=500, max_objects=5):
    """Filter and retain the largest root components."""
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    valid_objects = [(i, stats[i, cv2.CC_STAT_AREA]) for i in range(1, num_labels) if stats[i, cv2.CC_STAT_AREA] >= min_area]
    largest_objects = sorted(valid_objects, key=lambda x: x[1], reverse=True)[:max_objects]

    filtered_mask = np.zeros_like(mask, dtype=np.uint8)
    for obj_id, _ in largest_objects:
        filtered_mask[labels == obj_id] = 255
    return filtered_mask, largest_objects

def measure_bounding_boxes(filtered_mask):
    """Measure roots with bounding boxes."""
    contours, _ = cv2.findContours(filtered_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
    bounding_boxes = sorted(bounding_boxes, key=lambda b: b[0])
    return bounding_boxes

def find_lowest_point(mask, bounding_box):
    """Find the lowest point in a bounding box."""
    x, y, w, h = bounding_box
    roi = mask[y:y+h, x:x+w]
    coordinates = np.column_stack(np.where(roi > 0))
    if len(coordinates) == 0:
        return None
    lowest_local = coordinates[np.argmax(coordinates[:, 0])]
    return lowest_local + [y, x]

def convert_to_mm(pixel_coords, image_shape, plate_size_mm=150):
    """Convert pixel coordinates to mm-space."""
    h, w = image_shape
    conversion_factor_x = plate_size_mm / w
    conversion_factor_y = plate_size_mm / h
    return np.array([pixel_coords[1] * conversion_factor_x, pixel_coords[0] * conversion_factor_y])

# Directory containing images
input_dir = "kaggle_test"
model_path = "232430_unet_model_128px_v7md.keras"
plate_position_robot = np.array([0.10775, 0.088 - 0.026, 0.057])  # Adjust based on your setup

# Load the model
model = tf.keras.models.load_model(model_path)

# Process all images in the directory
all_root_tip_coords = {}

for image_name in sorted(os.listdir(input_dir)):
    if not image_name.endswith((".png", ".jpg", ".jpeg")):
        continue

    image_path = os.path.join(input_dir, image_name)
    logging.info(f"Processing image: {image_name}")

    try:
        # Process the image and extract root tip coordinates
        image = preprocess_image(image_path)
        petri_dish = extract_petri_dish(image)
        predicted_mask = predict_root_mask(petri_dish, model)
        connected_mask = connect_roots(predicted_mask)
        filtered_mask, _ = filter_and_select_largest_objects(connected_mask, min_area=500, max_objects=5)
        bounding_boxes = measure_bounding_boxes(filtered_mask)

        root_tip_coords = []
        for box in bounding_boxes:
            point = find_lowest_point(filtered_mask, box)
            if point is not None:
                mm_coords = convert_to_mm(point, petri_dish.shape)
                robot_coords = np.append(mm_coords, 0) + plate_position_robot
                root_tip_coords.append(robot_coords)

        all_root_tip_coords[image_name] = root_tip_coords
        logging.info(f"Extracted Root Tip Coordinates for {image_name}: {root_tip_coords}")

    except Exception as e:
        logging.error(f"Error processing image {image_name}: {e}")

# Expose all_root_tip_coords for import
def get_root_tip_coords():
    return all_root_tip_coords

# Log all extracted root tip coordinates
logging.info("All Extracted Root Tip Coordinates:")
for img_name, coords in all_root_tip_coords.items():
    logging.info(f"{img_name}: {coords}")


2025-01-09 08:34:51.538359: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2025-01-09 08:34:51.538749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 28317 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:c1:00.0, compute capability: 8.6
INFO: Processing image: test_image_1.png
2025-01-09 08:34:58.321900: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8906
INFO: Extracted Root Tip Coordinates for test_image_1.png: [array([ 6.89359458, 43.1754469 ,  0.057     ]), array([1.65251159e+01, 6.26390207e+01, 5.70000000e-02]), array([23.2015114 , 54.59262704,  0.057     ]), array([7.61201542e+01, 3.94784552e+01, 5.70000000e-02]), array([1.25864773e+02, 4.50239427e+01, 5.70000000e-02])]
INFO: Processing image: test_image_10.png
I

In [7]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import logging
from pid_class import PIDController
from ot2_class import OT2Env

# Logging configuration
log_file = "adaptive_pid_optimized_ot2.log"
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[logging.StreamHandler(), logging.FileHandler(log_file)],
)

logging.info("Starting Optimized PID Controller Test with OT2Env")

# Initialize the OT2 environment
env = OT2Env()

# Initialize PID controllers with dynamic integral limits
pid_x = PIDController(kp=1.0, ki=0.1, kd=0.05)  # Example values
pid_y = PIDController(kp=1.0, ki=0.1, kd=0.05)
pid_z = PIDController(kp=1.0, ki=0.1, kd=0.05)


# Simplified PID gain adjustment
def adjust_pid_gains(pid_x, pid_y, pid_z, error):
    if error > 0.5:
        pid_x.Kp, pid_x.Ki, pid_x.Kd = 8, 0.1, 3
        pid_y.Kp, pid_y.Ki, pid_y.Kd = 8, 0.1, 3
        pid_z.Kp, pid_z.Ki, pid_z.Kd = 6, 0.1, 2.5
    elif 0.2 < error <= 0.5:
        pid_x.Kp, pid_x.Ki, pid_x.Kd = 6, 0.08, 2.5
        pid_y.Kp, pid_y.Ki, pid_y.Kd = 6, 0.08, 2.5
        pid_z.Kp, pid_z.Ki, pid_z.Kd = 5, 0.08, 2
    else:
        pid_x.Kp, pid_x.Ki, pid_x.Kd = 4, 0.05, 1.5
        pid_y.Kp, pid_y.Ki, pid_y.Kd = 4, 0.05, 1.5
        pid_z.Kp, pid_z.Ki, pid_z.Kd = 3, 0.05, 1.2

# Function to dynamically initialize closer to target
def dynamic_initialize(target_position):
    # Clip to ensure within simulation range and convert to float32
    return np.clip(target_position, -0.3, 0.3).astype(np.float32)

# Get coordinates from the pipeline
root_tip_coords = all_root_tip_coords

if not root_tip_coords:
    logging.error("No root tip coordinates provided.")
    print("No root tip coordinates found. Exiting...")
    exit()

# Iterate over each image and its root tip coordinates
for image_name, coordinates in root_tip_coords.items():
    logging.info(f"Processing root tips for {image_name}")
    print(f"Processing {image_name}...")

    for idx, target_position in enumerate(coordinates):
        # Ensure target position is float32
        target_position = np.array(target_position, dtype=np.float32)
        logging.info(f"Root Tip {idx + 1}: Target Position: {target_position}")
        print(f"Root Tip {idx + 1}: Target Position: {target_position}")

        try:
            # Dynamic initialization
            initial_position = dynamic_initialize(target_position)
            observation, info = env.reset()
            print(f"Environment reset. Initial observation: {observation}")
        except Exception as e:
            logging.error(f"Environment reset failed: {e}")
            print(f"Failed to reset environment for {image_name}, Root Tip {idx + 1}.")
            continue

        # Initialize loop variables
        terminated = False
        epoch = 0
        recent_errors = []
        all_positions = []

        # Simulation loop
        while not terminated and epoch < 200:
            epoch += 1

            # Current pipette position
            current_position = observation[:3]
            all_positions.append(current_position)

            # Compute individual axis errors
            error = np.linalg.norm(target_position - current_position)

            # Log errors and coordinates
            logging.info(f"Epoch: {epoch} | Error: {error:.4f} | Current Position: "
                         f"X={current_position[0]:.4f}, Y={current_position[1]:.4f}, Z={current_position[2]:.4f}")

            # Success criterion
            if error <= 0.005:
                logging.info(f"Root Tip {idx + 1}: Target reached successfully in {epoch} epochs. Final Error: {error:.4f}")
                print(f"Root Tip {idx + 1} reached in {epoch} epochs with final error {error:.4f}")
                break

            # Dynamically adjust PID gains based on error magnitude
            adjust_pid_gains(pid_x, pid_y, pid_z, error)

            # Update integral limit based on error magnitude
            pid_x.integral_limit = pid_y.integral_limit = max(1.0, error * 2)
            pid_z.integral_limit = max(0.5, error)

            # Compute control actions with action saturation
            control_x = np.clip(pid_x.compute(current_position[0]), -1, 1)
            control_y = np.clip(pid_y.compute(current_position[1]), -1, 1)
            control_z = np.clip(pid_z.compute(current_position[2]), -1, 1)
            action = np.array([control_x, control_y, control_z])

            # Take a step in the environment
            try:
                observation, _, terminated, truncated, _ = env.step(action)
            except Exception as e:
                logging.error(f"Environment step failed: {e}")
                print(f"Failed to take a step for {image_name}, Root Tip {idx + 1}.")
                break

            # Track recent errors for early termination
            recent_errors.append(error)
            if len(recent_errors) > 20:
                avg_error_change = abs(recent_errors[-1] - recent_errors[0]) / 20
                if avg_error_change < 0.0001:
                    print(f"Early termination at epoch {epoch} due to negligible error change.")
                    break

            # Handle truncation
            if truncated:
                logging.warning("Environment truncated. Resetting...")
                observation, info = env.reset()

        # Log final result for this root tip
        if error > 0.005:
            logging.warning(f"Root Tip {idx + 1}: Failed to reach target within 200 epochs.")
            print(f"Root Tip {idx + 1} failed to reach target within 200 epochs.")

        # Visualize trajectories
        target_x, target_y, target_z = target_position
        actual_positions = np.array(all_positions)
        plt.figure()
        plt.plot(actual_positions[:, 0], actual_positions[:, 1], label="Trajectory")
        plt.scatter([target_x], [target_y], color="red", label="Target", marker="x")
        plt.xlabel("X Position")
        plt.ylabel("Y Position")
        plt.title(f"Trajectory for Root Tip {idx + 1}")
        plt.legend()
        plt.savefig(f"trajectory_root_tip_{idx + 1}.png")
        plt.close()

logging.info("PID Controller Test Complete.")
print("PID Controller Test Complete. Check the log file for details.")


INFO: Starting Optimized PID Controller Test with OT2Env
INFO: Processing root tips for test_image_1.png
INFO: Root Tip 1: Target Position: [ 6.8935947 43.175446   0.057    ]
INFO: Epoch: 1 | Error: 43.6225 | Current Position: X=0.0730, Y=0.0895, Z=0.1195
INFO: Epoch: 2 | Error: 43.6250 | Current Position: X=0.0706, Y=0.0874, Z=0.1199
INFO: Epoch: 3 | Error: 43.6265 | Current Position: X=0.0653, Y=0.0867, Z=0.1197
INFO: Epoch: 4 | Error: 43.6276 | Current Position: X=0.0620, Y=0.0861, Z=0.1195
INFO: Epoch: 5 | Error: 43.6286 | Current Position: X=0.0581, Y=0.0857, Z=0.1193
INFO: Epoch: 6 | Error: 43.6294 | Current Position: X=0.0557, Y=0.0853, Z=0.1191
INFO: Epoch: 7 | Error: 43.6306 | Current Position: X=0.0534, Y=0.0844, Z=0.1189
INFO: Epoch: 8 | Error: 43.6309 | Current Position: X=0.0504, Y=0.0846, Z=0.1187


Processing test_image_1.png...
Root Tip 1: Target Position: [ 6.8935947 43.175446   0.057    ]
Environment reset. Initial observation: [ 0.073       0.0895      0.1195     -0.09238425  0.11135813 -0.01377117]


INFO: Epoch: 9 | Error: 43.6308 | Current Position: X=0.0501, Y=0.0847, Z=0.1184
INFO: Epoch: 10 | Error: 43.6304 | Current Position: X=0.0503, Y=0.0851, Z=0.1182
INFO: Epoch: 11 | Error: 43.6301 | Current Position: X=0.0505, Y=0.0854, Z=0.1179
INFO: Epoch: 12 | Error: 43.6291 | Current Position: X=0.0508, Y=0.0863, Z=0.1177
INFO: Epoch: 13 | Error: 43.6280 | Current Position: X=0.0512, Y=0.0874, Z=0.1175
INFO: Epoch: 14 | Error: 43.6282 | Current Position: X=0.0483, Y=0.0877, Z=0.1174
INFO: Epoch: 15 | Error: 43.6287 | Current Position: X=0.0460, Y=0.0875, Z=0.1171
INFO: Epoch: 16 | Error: 43.6283 | Current Position: X=0.0431, Y=0.0884, Z=0.1170
INFO: Epoch: 17 | Error: 43.6281 | Current Position: X=0.0425, Y=0.0887, Z=0.1166
INFO: Epoch: 18 | Error: 43.6284 | Current Position: X=0.0419, Y=0.0885, Z=0.1163
INFO: Epoch: 19 | Error: 43.6279 | Current Position: X=0.0419, Y=0.0890, Z=0.1157
INFO: Epoch: 20 | Error: 43.6273 | Current Position: X=0.0415, Y=0.0896, Z=0.1153
INFO: Epoch: 21 |

Early termination at epoch 49 due to negligible error change.
Root Tip 1 failed to reach target within 200 epochs.
Root Tip 2: Target Position: [1.6525116e+01 6.2639019e+01 5.7000000e-02]
Environment reset. Initial observation: [0.073      0.0895     0.1195     0.18939573 0.28316623 0.29877695]


INFO: Epoch: 5 | Error: 64.6805 | Current Position: X=0.0683, Y=0.0871, Z=0.1204
INFO: Epoch: 6 | Error: 64.6809 | Current Position: X=0.0681, Y=0.0868, Z=0.1201
INFO: Epoch: 7 | Error: 64.6813 | Current Position: X=0.0675, Y=0.0865, Z=0.1198
INFO: Epoch: 8 | Error: 64.6818 | Current Position: X=0.0661, Y=0.0864, Z=0.1195
INFO: Epoch: 9 | Error: 64.6821 | Current Position: X=0.0657, Y=0.0862, Z=0.1191
INFO: Epoch: 10 | Error: 64.6825 | Current Position: X=0.0648, Y=0.0860, Z=0.1187
INFO: Epoch: 11 | Error: 64.6828 | Current Position: X=0.0641, Y=0.0859, Z=0.1183
INFO: Epoch: 12 | Error: 64.6831 | Current Position: X=0.0637, Y=0.0857, Z=0.1180
INFO: Epoch: 13 | Error: 64.6835 | Current Position: X=0.0631, Y=0.0854, Z=0.1176
INFO: Epoch: 14 | Error: 64.6835 | Current Position: X=0.0627, Y=0.0855, Z=0.1173
INFO: Epoch: 15 | Error: 64.6833 | Current Position: X=0.0625, Y=0.0858, Z=0.1169
INFO: Epoch: 16 | Error: 64.6835 | Current Position: X=0.0625, Y=0.0856, Z=0.1166
INFO: Epoch: 17 | Err

...

Root Tip 3 failed to reach target within 200 epochs.
Root Tip 4: Target Position: [1.0609417e+02 5.0921291e+01 5.7000000e-02]
Environment reset. Initial observation: [0.073      0.0895     0.1195     0.2749168  0.29225445 0.27877095]


INFO: Epoch: 6 | Error: 117.5735 | Current Position: X=0.0768, Y=0.0899, Z=0.1193
INFO: Epoch: 7 | Error: 117.5723 | Current Position: X=0.0777, Y=0.0908, Z=0.1192
INFO: Epoch: 8 | Error: 117.5711 | Current Position: X=0.0785, Y=0.0918, Z=0.1191
INFO: Epoch: 9 | Error: 117.5702 | Current Position: X=0.0790, Y=0.0928, Z=0.1191
INFO: Epoch: 10 | Error: 117.5697 | Current Position: X=0.0795, Y=0.0930, Z=0.1190
INFO: Epoch: 11 | Error: 117.5694 | Current Position: X=0.0799, Y=0.0927, Z=0.1189
INFO: Epoch: 12 | Error: 117.5695 | Current Position: X=0.0801, Y=0.0921, Z=0.1189
INFO: Epoch: 13 | Error: 117.5696 | Current Position: X=0.0802, Y=0.0918, Z=0.1189
INFO: Epoch: 14 | Error: 117.5694 | Current Position: X=0.0804, Y=0.0918, Z=0.1189
INFO: Epoch: 15 | Error: 117.5688 | Current Position: X=0.0807, Y=0.0926, Z=0.1188
INFO: Epoch: 16 | Error: 117.5681 | Current Position: X=0.0813, Y=0.0928, Z=0.1187
INFO: Epoch: 17 | Error: 117.5678 | Current Position: X=0.0819, Y=0.0924, Z=0.1186
INFO: Ep

Early termination at epoch 183 due to negligible error change.
Root Tip 4 failed to reach target within 200 epochs.
Root Tip 5: Target Position: [1.3193405e+02 6.0749432e+01 5.7000000e-02]
Environment reset. Initial observation: [ 0.073       0.0895      0.1195      0.26292476 -0.09462339 -0.07886596]


INFO: Epoch: 6 | Error: 145.1431 | Current Position: X=0.0770, Y=0.0845, Z=0.1193
INFO: Epoch: 7 | Error: 145.1431 | Current Position: X=0.0773, Y=0.0839, Z=0.1192
INFO: Epoch: 8 | Error: 145.1425 | Current Position: X=0.0783, Y=0.0831, Z=0.1191
INFO: Epoch: 9 | Error: 145.1425 | Current Position: X=0.0787, Y=0.0823, Z=0.1190
INFO: Epoch: 10 | Error: 145.1423 | Current Position: X=0.0790, Y=0.0821, Z=0.1188
INFO: Epoch: 11 | Error: 145.1423 | Current Position: X=0.0793, Y=0.0814, Z=0.1186
INFO: Epoch: 12 | Error: 145.1422 | Current Position: X=0.0799, Y=0.0803, Z=0.1184
INFO: Epoch: 13 | Error: 145.1421 | Current Position: X=0.0807, Y=0.0789, Z=0.1183
INFO: Epoch: 14 | Error: 145.1424 | Current Position: X=0.0809, Y=0.0776, Z=0.1180
INFO: Epoch: 15 | Error: 145.1427 | Current Position: X=0.0813, Y=0.0761, Z=0.1178
INFO: Epoch: 16 | Error: 145.1430 | Current Position: X=0.0815, Y=0.0749, Z=0.1176
INFO: Epoch: 17 | Error: 145.1429 | Current Position: X=0.0820, Y=0.0740, Z=0.1175
INFO: Ep

Early termination at epoch 21 due to negligible error change.
Root Tip 5 failed to reach target within 200 epochs.
PID Controller Test Complete. Check the log file for details.
